### replace default value of argument (e.g. `gamma`, `hi_min`) to defined values in module file

### Test for several set of initial values

## How can I find those kind of `t0+1/x0` singular point .. where propagation is not possible .. although,, in Bohm, there is always solution if started from regular point -- thus, small enough stepsize always exist to

# compare implicit and explicit Euler

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['font.size'] = 15

In [2]:
from bohm.ode.euler import get_stepsize, back_euler

from bohm.sampling import quantile

In [132]:
from tdse.analytic.state import Gaussian1D, gradient_Gaussian1D, laplacian_Gaussian1D

A1, A2 = 1.0, 0.8
kx1, kx2 = 0.0, 0.0
xc1, xc2 = -2.0, 2.0
t01, t02 = 0.0, 0.1
def psi(t,x): return A1 * Gaussian1D(x-xc1,t-t01,kx1) + A2 * Gaussian1D(x-xc2,t-t02,kx2)
def dxpsi(t,x): return A1 * gradient_Gaussian1D(x-xc1,t-t01,kx1) + A2 * gradient_Gaussian1D(x-xc2,t-t02,kx2)
def dx2psi(t,x): return A1 * laplacian_Gaussian1D(x-xc1,t-t01,kx1) + A2 * laplacian_Gaussian1D(x-xc2,t-t02,kx2)

def Fx(t,x): return (dxpsi(t,x) / psi(t,x)).imag
def dxFx(t,x): return (dx2psi(t,x)/psi(t,x) - (dxpsi(t,x)/psi(t,x))**2).imag

t0 = -3
t_max = 3

x_arr = np.linspace(-8,8,71)

quant_arr = np.linspace(0.05, 0.95, 21)
prob0_arr = np.square(np.abs(psi(t0, x_arr)))
x0_arr_pool = quantile(x_arr, prob0_arr, quant_arr)
# x0_arr = np.linspace(-3, 1, 30)

x0_arr = x0_arr_pool[np.abs(psi(t0,x0_arr_pool)) > 0.4]
assert np.all(np.abs(psi(t0, x0_arr)) != 0)
print("number of particles: {}".format(x0_arr.size))

number of particles: 42


In [133]:
t_arr = np.linspace(t0, t_max, 30)
x_t_arr = np.empty((t_arr.size, x0_arr.size), dtype=float)
x_t_arr[0,:] = x0_arr

h_max, gamma, h_min = 0.005, 0.5, 0.0001

for i in range(x0_arr.size):
    _, xp_arr = back_euler(t0, x0_arr[i], t_max, Fx, dxFx, h_max, 
                          t_arr=t_arr, gamma=gamma, hi_min=h_min)
    x_t_arr[:,i] = xp_arr

_i: 16, _h: 0.005, _t: 0.2784482758620691, _t_next: 0.2834482758620691, _t_target: 0.31034482758620685
t0: -3, x0: 4.961409271193066, _x: 2.2875344452911897
Fx: 0.0951829848169272, dxFx: 0.33110212304695175
sol:      fjac: array([[-1.]])
     fun: array([1.0467972e-16])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 14
     qtf: array([-1.0467972e-16])
       r: array([-1.11022302e-16])
  status: 5
 success: False
       x: array([2.28802255])


Exception: unique root search failed due to 'The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.'

In [ ]:
# figs, axs = plt.subplots()

# # x_arr = np.linspace(x0_arr.min(), x0_arr.max())
# axs.plot(x_arr, np.square(np.abs(psi(t0, x_arr))))
# axs.vlines(x0_arr, 0, 1)

In [ ]:
from vis.plot import construct_catesian_mesh_for_pcolormesh

# xt_min, xt_max = x_t_arr.min(), x_t_arr.max()
# xt_len = xt_max - xt_min
# xlim = (xt_min - 0.2 * xt_len, xt_max + 0.2 * xt_len)

xlim = (x_arr[0], x_arr[-1])

t_grid_vals = np.linspace(t0, t_max, 50)
x_grid_vals = np.linspace(*xlim, 50)
T_grid, X_grid = construct_catesian_mesh_for_pcolormesh(t_grid_vals, x_grid_vals)
T, X = np.meshgrid(t_grid_vals, x_grid_vals, indexing='ij')

In [ ]:
from matplotlib.colors import LogNorm, Normalize

psi_val = psi(T, X)
C = np.square(np.abs(psi_val))
vmax = C.max()
vmin = vmax * pow(10, -1.4)
# vmin = None

# norm = LogNorm(vmin, vmax)
norm = Normalize(vmin, vmax)

# [DONE] Implement sampling from the initial distribution

# Optimize each routine, especially in terms of derivative evaluation

In [ ]:
from matplotlib import cm

fig, ax = plt.subplots()

ax.contourf(X_grid[:-1,:-1], T_grid[:-1,:-1], C, 
            cmap=cm.jet, levels=cm.jet.N, norm=norm)

for i in range(x0_arr.size):
    l_num, = ax.plot(x_t_arr[:,i], t_arr, '.-w', linewidth=0.5)

ax.set_ylim(T_grid[:-1,:-1].min(), T_grid[:-1,:-1].max())
ax.set_xlim(X_grid[:-1,:-1].min(), X_grid[:-1,:-1].max())

ax.set_title("`h_max`={}, `gamma`={}".format(h_max, gamma))
ax.set_ylabel("time / a.u.")
ax.set_xlabel("x / a.u.")

In [ ]:
fig_fname = "step-esti-two-gaussian-A1-{}-A2-{}-t01-{}-t02-{}-hmax-{}-gamma-{}-with-map-test-2.png".format(
    A1, A2, t01, t02, h_max,gamma)
print(fig_fname)
# fig.savefig(fig_fname)